# Clasificación binaria
Se implementará un modelo de clasificación binaria para un datasets con clases altamente desbalanceadas.

## Descargar el dataset

In [1]:
!curl -O https://storage.googleapis.com/naylalex-deep-learning/Kaggle-Credit-Card-Fraud-Detection.zip
!unzip -d datasets/credit-card-fraud-detection/ Kaggle-Credit-Card-Fraud-Detection.zip
!rm Kaggle-Credit-Card-Fraud-Detection.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  1 65.9M    1 1033k    0     0   902k      0  0:01:14  0:00:01  0:01:13  921k
  3 65.9M    3 2494k    0     0  1169k      0  0:00:57  0:00:02  0:00:55 1182k
  5 65.9M    5 3950k    0     0  1240k      0  0:00:54  0:00:03  0:00:51 1249k
  7 65.9M    7 4958k    0     0  1193k      0  0:00:56  0:00:04  0:00:52 1200k
  9 65.9M    9 6606k    0     0  1286k      0  0:00:52  0:00:05  0:00:47 1320k
 12 65.9M   12 8622k    0     0  1407k      0  0:00:47  0:00:06  0:00:41 1523k
 14 65.9M   14 9902k    0     0  1389k      0  0:00:48  0:00:07  0:00:41 1483k
 16 65.9M   16 10.6M    0     0  1342k      0  0:00:50  0:00:08  0:00:42 1408k
 18 65.9M   18 12.3M    0     0  1382k      0  0:00

## Vectorizar los datos CSV

In [1]:
import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "./datasets/credit-card-fraud-detection/creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


## Preparar un dataset de validación

In [4]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


## Analizar el desbalance de clases en los targets

In [5]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


## Normalizar los datos empleando estadísticas del training set

In [6]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

## Generar un modelo de clasificación binaria

In [7]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               7936      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 139777 (546.00 KB)
Trainable params: 13977

## Entrenar el modelo con el argumento class_weight

In [9]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("logs/fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Epoch 1/30
112/112 - 7s - loss: 1.9409e-07 - fn: 0.0000e+00 - fp: 2757.0000 - tn: 224672.0000 - tp: 417.0000 - precision: 0.1314 - recall: 1.0000 - val_loss: 0.0114 - val_fn: 9.0000 - val_fp: 205.0000 - val_tn: 56681.0000 - val_tp: 66.0000 - val_precision: 0.2435 - val_recall: 0.8800 - 7s/epoch - 60ms/step
Epoch 2/30


c:\Users\Marciano\.conda\envs\class\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


112/112 - 4s - loss: 1.0067e-07 - fn: 0.0000e+00 - fp: 1191.0000 - tn: 226238.0000 - tp: 417.0000 - precision: 0.2593 - recall: 1.0000 - val_loss: 0.0071 - val_fn: 14.0000 - val_fp: 89.0000 - val_tn: 56797.0000 - val_tp: 61.0000 - val_precision: 0.4067 - val_recall: 0.8133 - 4s/epoch - 38ms/step
Epoch 3/30
112/112 - 4s - loss: 1.2996e-07 - fn: 1.0000 - fp: 1274.0000 - tn: 226155.0000 - tp: 416.0000 - precision: 0.2462 - recall: 0.9976 - val_loss: 0.0077 - val_fn: 14.0000 - val_fp: 83.0000 - val_tn: 56803.0000 - val_tp: 61.0000 - val_precision: 0.4236 - val_recall: 0.8133 - 4s/epoch - 39ms/step
Epoch 4/30
112/112 - 4s - loss: 1.1192e-07 - fn: 0.0000e+00 - fp: 1118.0000 - tn: 226311.0000 - tp: 417.0000 - precision: 0.2717 - recall: 1.0000 - val_loss: 0.0415 - val_fn: 10.0000 - val_fp: 432.0000 - val_tn: 56454.0000 - val_tp: 65.0000 - val_precision: 0.1308 - val_recall: 0.8667 - 4s/epoch - 37ms/step
Epoch 5/30
112/112 - 6s - loss: 2.3498e-07 - fn: 1.0000 - fp: 2139.0000 - tn: 225290.0000 

## Conclusiones
* Número de transacciones de validación.
* Número de casos identificados correctamente como fraudulentos.
* Número de transacciones fraudulentas faltantes.
* Costo de etiquetar 441 transacciones legítimas.
Recomendaciones: Colocar un peso mayor en la clase 1 para reflejar que los falsos negativos son más costosos que los falsos positivos.
